In [1]:
#!pip install -q torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 torchtext==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
#!pip install -q --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install torch-geometric
#!pip install optuna

In [3]:
import torch
import optuna
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from torch_geometric.data import NeighborSampler
from torch.optim import lr_scheduler
import torch.optim as optim
import matplotlib.pyplot as plt
import collections
import numpy as np
from modules.model import Net
from sklearn.metrics import f1_score
from modules.sampling import Sampler, SamplerContextMatrix, SamplerRandomWalk,SamplerFactorization,SamplerAPP
from datetime import datetime
import random
from torch_geometric.data import GraphSAINTNodeSampler
import pickle

In [4]:
from torch_geometric.datasets import Amazon
flag = False
dataset =Planetoid(root='/tmp/Pubmed', name='Pubmed',transform=T.NormalizeFeatures())
#dataset=Amazon(root='/tmp/Photo', name='Photo',transform=T.NormalizeFeatures())

if flag: 
    data = dataset[0]

    indices_sh=list(range(len(data.x)))
    indices=list(range(len(data.x)))
    random.shuffle(indices_sh)
    x_new_map={}
    for i,x in enumerate(indices_sh):
        x_new_map[x] = i
    data_x_new = data.x[indices_sh]
    data_y_new = data.x[indices_sh]
    data_edge_index = torch.tensor(list(map(lambda x: [x_new_map[int(x[0])],x_new_map[int(x[1])]] ,data.edge_index.t()))).t()

    data.x=data_x_new
    data.edge_index=data_edge_index
    data.y=data_y_new
    with open('data_pubmed_new.pickle','wb') as f:
        pickle.dump(data,f)
else:
    with open('data_pubmed_new.pickle','rb') as f:
        data = pickle.load(f)
print((data.x).shape)
indices=list(range(len(data.x)))
train_indices = torch.tensor(indices[:int(0.7*len(indices)+1)])
val_indices = torch.tensor(indices[int(0.7*len(indices)+1):int(0.8*len(indices)+1)])
test_indices = torch.tensor(indices[int(0.8*len(indices)+1):])
train_mask = torch.tensor([False]*len(indices))
test_mask = torch.tensor([False]*len(indices))
val_mask = torch.tensor([False]*len(indices))
train_mask[train_indices] =True
test_mask[test_indices]=True
val_mask[val_indices]=True
        
#data_Computers =Amazon(root='/tmp/Computers', name='Computers',transform=T.NormalizeFeatures())[0]

torch.Size([19717, 500])


In [5]:
import pickle
import os

class Main():
    def __init__(self,conv, device, loss_function, mode ,dataset,data,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask):
        #self.dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
        self.dataset=dataset
        data = self.dataset[0]
        
        self.Conv = conv
        self.device = device
        self.dataset = dataset
        self.x = data.x
        self.y = data.y.squeeze()
        self.data=data.to(device)
        self.loss = loss_function
        self.mode = mode
        self.train_indices =train_indices# torch.tensor(indices[:int(0.7*len(indices)+1)])
        self.val_indices =val_indices# torch.tensor(indices[int(0.7*len(indices)+1):int(0.8*len(indices)+1)])
        self.test_indices = test_indices#torch.tensor(indices[int(0.8*len(indices)+1):])
        self.train_mask = train_mask#torch.tensor([False]*len(indices))
        self.test_mask = test_mask#torch.tensor([False]*len(indices))
        self.val_mask =val_mask# torch.tensor([False]*len(indices))
        self.flag = self.loss["flag_tosave"]
        #print('huhu',self.data.x.shape)
        super(Main, self).__init__()
    def sampling(self,Sampler,epoch,nodes):
        if (epoch == 0): 
            if self.flag:  
                if "alpha" in self.loss: 
                    name_of_file = "samples_"+self.loss["Name"]+"_alpha_"+str(self.loss["alpha"])+".pickle"
                else:
                    name_of_file = "samples_"+self.loss["Name"]+".pickle"
                
                if os.path.exists(name_of_file):
                    with open(name_of_file,'rb') as f:
                        self.samples = pickle.load(f)
                else:
                    self.samples = Sampler.sample(nodes) 
                    with open(name_of_file,'wb') as f:
                        pickle.dump(self.samples,f)
            else:
                
                self.samples = Sampler.sample(nodes)
 
    def train(self, model,data,optimizer,Sampler,train_loader,dropout,epoch):
        model.train()   
        total_loss = 0
        optimizer.zero_grad()
        if model.mode == 'unsupervised':
            if model.conv=='GCN':
                arr=torch.nonzero(self.train_mask == True)
                indices_of_train_data = ([item for sublist in arr for item in sublist])
                out = model.inference(data.to(device),dp=dropout)
                samples = self.sampling(Sampler,epoch, indices_of_train_data)
                loss = model.loss(out[self.train_mask], self.samples)
                total_loss+=loss
            else:
                for batch_size, n_id, adjs in train_loader:
                    #if len(train_loader.sizes) == 1:
                     #   adjs = [adjs]
                    adjs = [adj.to(device) for adj in adjs]
                    out = model.forward(data.x[n_id.to(device)].to(device), adjs)
                    self.sampling(Sampler,epoch,n_id[:batch_size])                 
                    loss = model.loss(out, self.samples)#pos_batch.to(device), neg_batch.to(device))
                    total_loss+=loss
            total_loss.backward()
            optimizer.step()      
            return total_loss /len(train_loader)
        elif model.mode== 'supervised':
            if model.conv=='GCN':
                out = model.inference(data.to(device),dp=dropout)
                loss = model.loss_sup(out[self.train_mask],y[self.train_mask])
                total_loss+=loss
            else:
                for batch_size, n_id, adjs in train_loader:
                    adjs = [adj for adj in adjs]
                    out = model.forward(data.x[n_id].to(device), adjs)
                    loss = model.loss_sup(out,y[n_id[:batch_size]])
                    total_loss+=loss
            total_loss.backward(retain_graph=True)
            optimizer.step()      
            return total_loss /len(train_loader)       

    @torch.no_grad()
    def test(self, model,data,classifier,**kwargs):#,n_estimators,learning_rate_carboost, max_depth): 
        model.eval()
        out = model.inference(data.to(device))
        y_true = self.y.detach().numpy()
        if model.mode == 'supervised':
            y_true = self.y.unsqueeze(-1)
            y_pred = out.cpu().argmax(dim=-1, keepdim=True)
            accs = []
            for mask in [self.train_mask, self.val_mask, self.test_mask]:    
                accs+=[int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]
            return accs
        elif model.mode == 'unsupervised': 
            if classifier == 'logistic regression':
                clf = LogisticRegression(max_iter = 3000,warm_start=True).fit(out.cpu().detach()[self.train_mask].numpy(), self.y.detach()[self.train_mask].numpy())
            else:
                n_estimators= kwargs["n_estimators"]
                learning_rate_catboost = kwargs["learning_rate_catboost"]
                max_depth = kwargs["max_depth"]
                clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate = learning_rate_catboost, max_depth=max_depth, random_state=0)
                clf.fit(out.cpu().detach()[self.train_mask].numpy(), self.y.detach()[self.train_mask].numpy())
            accs_micro = []
            accs_macro = []
            for mask in [self.train_mask,self.test_mask,self.val_mask]:
                accs_micro += [f1_score(self.y.detach()[mask].cpu().numpy(),clf.predict(out.cpu().detach()[mask].numpy()), average='micro')]
                accs_macro += [f1_score(self.y.detach()[mask].cpu().numpy(),clf.predict(out.cpu().detach()[mask].numpy()), average='macro')]
                
            return accs_micro,accs_macro

    def run(self,hidden_layer=64,out_layer=128,dropout=0.0,size=1,learning_rate=0.001):
        

        classifier = "logistic regression"
        train_loader = NeighborSampler(self.data.edge_index, node_idx=self.train_mask, batch_size = int(sum(self.train_mask)), sizes=[-1]*size)
        Sampler = self.loss["Sampler"]
        LossSampler = Sampler(self.data,device=device,mask=self.train_mask,loss_info=self.loss)
        model = Net(dataset = self.dataset,mode=self.mode,conv=self.Conv,loss_function=self.loss,device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = (size),dropout = dropout)
        model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)
                #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
        scheduler=lr_scheduler.StepLR(optimizer, step_size=25,gamma=0.1)
        losses=[]
        train_accs=[]
        test_accs=[]
        val_accs=[]
        name_of_plot='conv: '+model.conv+', mode: '+model.mode+', loss from '+self.loss["Name"]

        print(name_of_plot)

        for epoch in range(100):
                    print('epoch',epoch)
                    loss = self.train(model,self.data,optimizer,LossSampler,train_loader,dropout,epoch)
                    losses.append(loss)
                    [train_acc_mi, test_acc_mi,val_acc_mi],[train_acc_ma, test_acc_ma,val_acc_ma] = self.test(model,self.data,'logistic regression')
                    train_accs_mi.append(train_acc_mi)
                    test_accs_mi.append(test_acc_mi)
                    train_accs_mi.append(train_acc_ma)
                    test_accs_mi.append(test_acc_ma)
                    log = 'Loss: {:.4f}, Epoch: {:03d}, Train: {:.4f}, Test: {:.4f}'
                    #scheduler.step()
                    print(log.format(loss, epoch, train_acc, test_acc))
        print('Test acc on the last epoch ', test_acc)
        plt.plot(losses)
        plt.title(name_of_plot+' loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()
        plt.plot(test_accs_mi)
        plt.title(name_of_plot+' test f1 micro')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()
                  
        plt.plot(test_accs_ma)
        plt.title(name_of_plot+' test f1 macro')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()

In [6]:
class MainOptuna(Main):
    def objective(self,trial):
        # Integer parameter
        hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
        out_layer = trial.suggest_categorical("out_layer", [32,64,128])
        dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
        size = trial.suggest_categorical("size of network, number of convs", [1,2,3])
        Conv = self.Conv
        learning_rate= trial.suggest_float("lr",5e-4,1e-2)

        # варьируем параметры
        loss_to_train={}
        for name in self.loss:
            
            if type(self.loss[name]) == list :
                if len(self.loss[name]) == 3:
                    var = trial.suggest_int(name,self.loss[name][0],self.loss[name][1],step=self.loss[name][2])
                    loss_to_train[name] = var
                elif len(self.loss[name]) == 2:
                    var_2 = trial.suggest_float(name,self.loss[name][0],self.loss[name][1])
                    loss_to_train[name] = var_2
                else:
                    var_3 = trial.suggest_categorical(name, self.loss[name])
                    loss_to_train[name] = var_3
            else:
                loss_to_train[name] = self.loss[name]
        Sampler =loss_to_train["Sampler"]
        model = Net(dataset = self.dataset,mode='unsupervised',conv=Conv,loss_function=loss_to_train,device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = size,dropout = dropout)
        train_loader = NeighborSampler(self.data.edge_index, batch_size = int(sum(self.train_mask)),node_idx=self.train_mask, sizes=[-1]*size)
       # train_loader = NeighborSampler(self.data.edge_index, batch_size = int(sum(self.train_mask)), sizes=[-1]*size)
        LossSampler = Sampler(self.data,device=self.device,mask=self.train_mask,loss_info=loss_to_train)
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  

        classifier = "logistic regression" #trial.suggest_categorical("classifier", ["logistic regression", "catboost"])

        if classifier == "catboost":
            n_estimators = trial.suggest_int("n of estimators", 10,40,5)
            learning_rate_catboost = trial.suggest_float("lr_catboost",5e-4,1e-2)
            max_depth = trial.suggest_int("max_depth",1,10,2)
        else:
            n_estimators = -1
            learning_rate_catboost =-1
            max_depth = -1
        #training of the model
        for epoch in range(50):
            #print('hi',self.data.x.shape)
            loss = self.train(model,self.data,optimizer,LossSampler,train_loader,dropout,epoch)
            #self.test(model,self.data,classifier,n_estimators=n_estimators,learning_rate_catboost=learning_rate_catboost,max_depth=max_depth)
        [train_acc_mi, test_acc_mi,val_acc_mi], [train_acc_ma, test_acc_ma,val_acc_ma] = self.test(model,self.data,classifier,n_estimators=n_estimators,learning_rate_catboost=learning_rate_catboost,max_depth=max_depth)
        trial.report( np.sqrt(val_acc_mi*val_acc_ma) ,epoch)
        return np.sqrt(val_acc_mi*val_acc_ma)

    
    def run(self,number_of_trials):

        study = optuna.create_study(direction="maximize",study_name=self.loss["Name"]+" loss,"+str(self.Conv)+" conv")
        study.optimize(self.objective,n_trials = number_of_trials)

        print('Best trial:')
        trial = study.best_trial
        print(" Value: ", trial.value)
        print(" Params: ")
        for key, value in trial.params.items():
            print(" {}: {}".format(key,value))

In [7]:
from modules.sampling import Sampler, SamplerContextMatrix, SamplerRandomWalk,SamplerFactorization,SamplerAPP
#хорошо
DeepWalk = {"Name": "DeepWalk","walk_length":[5,20,5],"walks_per_node":[5,20,5],"num_negative_samples":[1,21,5],"context_size" : [5,20,5],"p":1,"q":1,"loss var": "Random Walks","flag_tosave":False,"Sampler" : SamplerRandomWalk } #Проблемы с памятью после того, как увеличила количество тренировочных данных
Node2Vec = {"Name": "Node2Vec","walk_length":[5,20,5],"walks_per_node":[5,20,5],"num_negative_samples":[1,21,5],"context_size" : [5,20,5],"p": [0.0,0.9] ,"q":[0.0,0.9], "loss var": "Random Walks","flag_tosave":False,"Sampler": SamplerRandomWalk}#то же самое 
LINE = {"Name": "LINE","C": "Adj","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":True,"Sampler" :SamplerContextMatrix} 

#плохо
HOPE_RPR = {"Name": "HOPE_RPR","C":"RPR","loss var": "Factorization","flag_tosave":False,"alpha": [0,1],"Sampler" :SamplerFactorization} #проверить
HOPE_Katz = {"Name": "HOPE_Katz","C":"Katz","loss var": "Factorization","flag_tosave":False,"betta": [0,1],"Sampler" :SamplerFactorization,} #проверить
HOPE_AA = {"Name": "HOPE_AdamicAdar","C":"AA","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization} 
LapEigen = {"Name": "LaplacianEigenMaps", "C":"Adj","loss var": "Laplacian EigenMaps","flag_tosave":True,"Sampler" :SamplerFactorization}
VERSE_PPR =  {"Name": "VERSE_PPR","C": "PPR","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerContextMatrix}

#под вопросом
VERSE_SR =  {"Name": "VERSE_SimRank","C": "SR","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":True,"Sampler":SamplerContextMatrix} 

Struc2Vec ={} #Implement
#в приципе можно найти нормальны результат через оптюну, но в основном плохо
GraphFactorization = {"Name": "Graph Factorization","C":"Adj","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization} 
HOPE_CN = {"Name": "HOPE_CommonNeighbors","C":"CN","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization} 
APP ={"Name": "APP","C": "PPR","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerAPP}
VERSE_Adj =  {"Name": "VERSE_Adj","C": "Adj","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"Sampler" :SamplerContextMatrix} 


In [8]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = APP
#pubmed
#print(data.x.shape)
MO = MainOptuna('GCN', device, loss , 'unsupervised',dataset,data,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=100)

[I 2021-05-03 14:54:13,849] A new study created in memory with name: APP loss,GCN conv
[I 2021-05-03 14:55:26,056] Trial 0 finished with value: 0.44266426296760586 and parameters: {'hidden_layer': 32, 'out_layer': 64, 'dropout': 0.1, 'size of network, number of convs': 3, 'lr': 0.001046964600316845, 'num_negative_samples': 6, 'alpha': 0.9}. Best is trial 0 with value: 0.44266426296760586.
[I 2021-05-03 14:58:40,043] Trial 1 finished with value: 0.5040832700503366 and parameters: {'hidden_layer': 128, 'out_layer': 64, 'dropout': 0.30000000000000004, 'size of network, number of convs': 3, 'lr': 0.0035373867374056036, 'num_negative_samples': 21, 'alpha': 0.8}. Best is trial 1 with value: 0.5040832700503366.
[I 2021-05-03 15:01:21,296] Trial 2 finished with value: 0.7067098001315859 and parameters: {'hidden_layer': 32, 'out_layer': 64, 'dropout': 0.0, 'size of network, number of convs': 2, 'lr': 0.007667242874355291, 'num_negative_samples': 11, 'alpha': 0.1}. Best is trial 2 with value: 0.

[I 2021-05-03 16:12:07,217] Trial 25 finished with value: 0.8092445800406817 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.008925699178996521, 'num_negative_samples': 1, 'alpha': 0.7}. Best is trial 23 with value: 0.8230919873421277.
[I 2021-05-03 16:13:14,239] Trial 26 finished with value: 0.7179538037181141 and parameters: {'hidden_layer': 32, 'out_layer': 32, 'dropout': 0.1, 'size of network, number of convs': 1, 'lr': 0.009106783461407642, 'num_negative_samples': 6, 'alpha': 0.7}. Best is trial 23 with value: 0.8230919873421277.
[I 2021-05-03 16:14:39,572] Trial 27 finished with value: 0.8085024882042573 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.30000000000000004, 'size of network, number of convs': 1, 'lr': 0.007847157090399012, 'num_negative_samples': 1, 'alpha': 0.7}. Best is trial 23 with value: 0.8230919873421277.
[I 2021-05-03 16:17:04,131] Trial 28 finished with value: 0.8199775

[I 2021-05-03 17:25:49,095] Trial 51 finished with value: 0.808530501152379 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.008113279030499722, 'num_negative_samples': 6, 'alpha': 0.1}. Best is trial 23 with value: 0.8230919873421277.
[I 2021-05-03 17:29:15,362] Trial 52 finished with value: 0.8044136138227503 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.008255411717560808, 'num_negative_samples': 6, 'alpha': 0.1}. Best is trial 23 with value: 0.8230919873421277.
[I 2021-05-03 17:32:42,059] Trial 53 finished with value: 0.8017019876362637 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.30000000000000004, 'size of network, number of convs': 1, 'lr': 0.006771168361871934, 'num_negative_samples': 6, 'alpha': 0.1}. Best is trial 23 with value: 0.8230919873421277.
[I 2021-05-03 17:36:47,547] Trial 54 finished with value: 0.8034

[I 2021-05-03 18:53:56,891] Trial 76 finished with value: 0.8130991278941142 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.00868968105602453, 'num_negative_samples': 21, 'alpha': 0.1}. Best is trial 23 with value: 0.8230919873421277.
[I 2021-05-03 18:59:29,545] Trial 77 finished with value: 0.7093982077669382 and parameters: {'hidden_layer': 32, 'out_layer': 64, 'dropout': 0.30000000000000004, 'size of network, number of convs': 2, 'lr': 0.009317333675636618, 'num_negative_samples': 16, 'alpha': 0.1}. Best is trial 23 with value: 0.8230919873421277.
[I 2021-05-03 19:04:00,333] Trial 78 finished with value: 0.8067979132577767 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.1, 'size of network, number of convs': 1, 'lr': 0.007922341781456784, 'num_negative_samples': 16, 'alpha': 0.5}. Best is trial 23 with value: 0.8230919873421277.
[I 2021-05-03 19:05:47,988] Trial 79 finished with value: 0.7329

Best trial:
 Value:  0.8230919873421277
 Params: 
 hidden_layer: 32
 out_layer: 128
 dropout: 0.30000000000000004
 size of network, number of convs: 1
 lr: 0.009117210746174291
 num_negative_samples: 6
 alpha: 0.6


In [9]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = APP
#pubmed
#print(data.x.shape)
MO = MainOptuna('GAT', device, loss , 'unsupervised',dataset,data,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=100)

[I 2021-05-03 20:19:03,383] A new study created in memory with name: APP loss,GAT conv
[I 2021-05-03 20:20:21,850] Trial 0 finished with value: 0.8080995782226056 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.00861570157646498, 'num_negative_samples': 1, 'alpha': 0.6}. Best is trial 0 with value: 0.8080995782226056.
[I 2021-05-03 20:22:20,780] Trial 1 finished with value: 0.7149515420671053 and parameters: {'hidden_layer': 32, 'out_layer': 32, 'dropout': 0.5, 'size of network, number of convs': 2, 'lr': 0.004291039179104722, 'num_negative_samples': 21, 'alpha': 0.2}. Best is trial 0 with value: 0.8080995782226056.
[I 2021-05-03 20:23:42,585] Trial 2 finished with value: 0.6858521402928395 and parameters: {'hidden_layer': 32, 'out_layer': 64, 'dropout': 0.30000000000000004, 'size of network, number of convs': 2, 'lr': 0.005770499211705641, 'num_negative_samples': 6, 'alpha': 0.6}. Best is trial 0 with value: 0.80809

[I 2021-05-03 21:38:19,818] Trial 26 finished with value: 0.7362626395304823 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 2, 'lr': 0.009992140058806711, 'num_negative_samples': 16, 'alpha': 0.9}. Best is trial 17 with value: 0.8114753899222809.
[I 2021-05-03 21:40:42,383] Trial 27 finished with value: 0.8055138339649859 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.30000000000000004, 'size of network, number of convs': 1, 'lr': 0.006506934811271764, 'num_negative_samples': 11, 'alpha': 0.9}. Best is trial 17 with value: 0.8114753899222809.
[I 2021-05-03 21:48:22,250] Trial 28 finished with value: 0.8010019424414752 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.007913007035655758, 'num_negative_samples': 21, 'alpha': 0.4}. Best is trial 17 with value: 0.8114753899222809.
[I 2021-05-03 21:56:15,929] Trial 29 finished with value: 0.804

[I 2021-05-03 23:15:16,467] Trial 52 finished with value: 0.8119887792728264 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008293318865323359, 'num_negative_samples': 16, 'alpha': 0.8}. Best is trial 50 with value: 0.8133963502365885.
[I 2021-05-03 23:18:29,523] Trial 53 finished with value: 0.8029999350876434 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.009856676951791949, 'num_negative_samples': 16, 'alpha': 0.8}. Best is trial 50 with value: 0.8133963502365885.
[I 2021-05-03 23:21:01,776] Trial 54 finished with value: 0.8112717263593605 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.009353631763634494, 'num_negative_samples': 11, 'alpha': 0.8}. Best is trial 50 with value: 0.8133963502365885.
[I 2021-05-03 23:24:15,163] Trial 55 finished with value: 0.8016352561734854

[I 2021-05-04 00:37:56,343] Trial 78 finished with value: 0.5923746872971918 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.30000000000000004, 'size of network, number of convs': 3, 'lr': 0.007995625327290394, 'num_negative_samples': 16, 'alpha': 0.8}. Best is trial 50 with value: 0.8133963502365885.
[I 2021-05-04 00:41:02,637] Trial 79 finished with value: 0.8029545976592326 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008477753306361123, 'num_negative_samples': 16, 'alpha': 0.9}. Best is trial 50 with value: 0.8133963502365885.
[I 2021-05-04 00:44:54,888] Trial 80 finished with value: 0.7936387165205688 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.30000000000000004, 'size of network, number of convs': 1, 'lr': 0.007727315523575522, 'num_negative_samples': 21, 'alpha': 0.9}. Best is trial 50 with value: 0.8133963502365885.
[I 2021-05-04 00:48:04,405] Trial 81 finished 

Best trial:
 Value:  0.8133963502365885
 Params: 
 hidden_layer: 128
 out_layer: 128
 dropout: 0.5
 size of network, number of convs: 1
 lr: 0.009532431483019818
 num_negative_samples: 16
 alpha: 0.8


In [10]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = APP
#pubmed
#print(data.x.shape)
MO = MainOptuna('SAGE', device, loss , 'unsupervised',dataset,data,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=100)

[I 2021-05-04 01:45:01,790] A new study created in memory with name: APP loss,SAGE conv
[I 2021-05-04 01:47:56,103] Trial 0 finished with value: 0.622587408750073 and parameters: {'hidden_layer': 128, 'out_layer': 64, 'dropout': 0.4, 'size of network, number of convs': 2, 'lr': 0.00147331734417405, 'num_negative_samples': 11, 'alpha': 0.1}. Best is trial 0 with value: 0.622587408750073.
[I 2021-05-04 01:50:07,235] Trial 1 finished with value: 0.7838478692736184 and parameters: {'hidden_layer': 32, 'out_layer': 64, 'dropout': 0.30000000000000004, 'size of network, number of convs': 1, 'lr': 0.00721857880847733, 'num_negative_samples': 11, 'alpha': 0.2}. Best is trial 1 with value: 0.7838478692736184.
[I 2021-05-04 01:51:12,168] Trial 2 finished with value: 0.8106796344395616 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.006990800072871003, 'num_negative_samples': 1, 'alpha': 0.8}. Best is trial 2 with value: 0.8106

[I 2021-05-04 02:46:26,007] Trial 26 finished with value: 0.8087618706250875 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.30000000000000004, 'size of network, number of convs': 1, 'lr': 0.006734420383537466, 'num_negative_samples': 1, 'alpha': 0.9}. Best is trial 14 with value: 0.817356022811754.
[I 2021-05-04 02:48:26,283] Trial 27 finished with value: 0.8106360654908122 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.008157033336121751, 'num_negative_samples': 6, 'alpha': 0.5}. Best is trial 14 with value: 0.817356022811754.
[I 2021-05-04 02:51:40,558] Trial 28 finished with value: 0.8116438402111713 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.009153056680787983, 'num_negative_samples': 16, 'alpha': 0.7}. Best is trial 14 with value: 0.817356022811754.
[I 2021-05-04 02:53:46,196] Trial 29 finished with value: 0.765356

[I 2021-05-04 03:56:18,989] Trial 52 finished with value: 0.8045587819200949 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008787453817747025, 'num_negative_samples': 1, 'alpha': 0.4}. Best is trial 14 with value: 0.817356022811754.
[I 2021-05-04 03:57:54,799] Trial 53 finished with value: 0.7942894092657871 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.004636473951823523, 'num_negative_samples': 6, 'alpha': 0.9}. Best is trial 14 with value: 0.817356022811754.
[I 2021-05-04 03:59:37,921] Trial 54 finished with value: 0.8093038057496085 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.007802996946827244, 'num_negative_samples': 6, 'alpha': 0.8}. Best is trial 14 with value: 0.817356022811754.
[I 2021-05-04 04:01:02,332] Trial 55 finished with value: 0.8028651898768094 and p

[I 2021-05-04 04:52:37,871] Trial 78 finished with value: 0.7335169035448539 and parameters: {'hidden_layer': 128, 'out_layer': 32, 'dropout': 0.0, 'size of network, number of convs': 1, 'lr': 0.009428384541012868, 'num_negative_samples': 6, 'alpha': 0.1}. Best is trial 62 with value: 0.8193086140369908.
[I 2021-05-04 04:56:12,749] Trial 79 finished with value: 0.6333133086796553 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.0, 'size of network, number of convs': 2, 'lr': 0.008562836425289759, 'num_negative_samples': 6, 'alpha': 0.1}. Best is trial 62 with value: 0.8193086140369908.
[I 2021-05-04 05:00:08,477] Trial 80 finished with value: 0.8085923164818835 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.0, 'size of network, number of convs': 1, 'lr': 0.00968485072596324, 'num_negative_samples': 11, 'alpha': 0.1}. Best is trial 62 with value: 0.8193086140369908.
[I 2021-05-04 05:03:20,405] Trial 81 finished with value: 0.817528675965934 and 

Best trial:
 Value:  0.8193086140369908
 Params: 
 hidden_layer: 256
 out_layer: 128
 dropout: 0.2
 size of network, number of convs: 1
 lr: 0.009294795724562429
 num_negative_samples: 6
 alpha: 0.8


In [11]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = GraphFactorization
#print(data.x.shape)
#pubmed
MO = MainOptuna('GCN', device, loss , 'unsupervised',dataset,data,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=500)

[I 2021-05-04 05:48:56,602] A new study created in memory with name: Graph Factorization loss,GCN conv
[I 2021-05-04 05:54:27,877] Trial 0 finished with value: 0.7487589807627462 and parameters: {'hidden_layer': 128, 'out_layer': 32, 'dropout': 0.5, 'size of network, number of convs': 2, 'lr': 0.0016278000428879547}. Best is trial 0 with value: 0.7487589807627462.
[I 2021-05-04 06:02:19,311] Trial 1 finished with value: 0.3110660727030963 and parameters: {'hidden_layer': 256, 'out_layer': 64, 'dropout': 0.1, 'size of network, number of convs': 3, 'lr': 0.00856817091727073}. Best is trial 0 with value: 0.7487589807627462.
[I 2021-05-04 06:09:54,519] Trial 2 finished with value: 0.44587670099180404 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.1, 'size of network, number of convs': 2, 'lr': 0.004720784697163159}. Best is trial 0 with value: 0.7487589807627462.
[I 2021-05-04 06:15:19,492] Trial 3 finished with value: 0.7314284963453886 and parameters: {'hidden_layer

KeyboardInterrupt: 

In [ ]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = GraphFactorization
#print(data.x.shape)
MO = MainOptuna('GAT', device, loss , 'unsupervised',dataset,data,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=500)

In [ ]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = GraphFactorization
#print(data.x.shape)
#pubmed
MO = MainOptuna('SAGE', device, loss , 'unsupervised',dataset,data,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=500)